In [ ]:
import math
import matplotlib.pyplot as plt
import scipy
import time

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
#sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(os.getcwd(), '..'))
from findOptShadingByGradDescent import *

In [ ]:
def this_d_oppo(x,K=3): return K*x**(K-1)
def this_cdf_oppo(x,K=3): return x**K
def unif_pdf(x): return 1.0
def unif_cdf(x): return x

In [ ]:
def max_unif_den(x,K=1):
    return K*x**(K-1)

def max_unif_cdf(x,K=1):
    return x**K

these_knots = [.25,.50,.75,1.0]
these_coeffs = [.3,.4,.1,.1,.1,.1]
this_x_0 = .6

In [ ]:
these_knots = np.linspace(.01,.99,50)
these_coeffs = np.zeros(len(these_knots)+2)
these_coeffs[0]=0
these_coeffs[1]=0.7
this_x_0 = .5
num_oppo = 1
num_steps = 300
grad_step = 10**(-1)
#I changed the code slightly there; the reward vector in the lp is now step_size*derivative_direction
start = time.time()
test, reserve_value_sequence, all_coeffs, reserve_prices, revenues=perform_multiple_gradient_ascent_steps(density_opponents= lambda x: max_unif_den(x,num_oppo), 
                                            cdf_opponents = lambda x: max_unif_cdf(x,num_oppo),
                                                 knots_bidder = these_knots,
                                                 coeffs_bidder_shading = these_coeffs,
                                                 density_bidder = max_unif_den, 
                                                cdf_bidder = max_unif_cdf,
                                                 x_0 = this_x_0, 
                                                assume_v_val_inc=False,
                                        lower_bound=0.0, upper_bound=1.0,
                                        gradient_step_size=grad_step,
                                        abs_max_size_move=1.0,
                                        number_steps=num_steps,
                                        less_than_id_constraint=False,
                                           monotonicity_constraint=False,
                                           brute_force_comp_res_val=True,
                                           test_vals=[0.0,1.0],
                                           my_tol = 10**(-7))
elapsed = time.time() - start
print("Elapsed time", elapsed)

In [ ]:
plt.plot(np.asarray(reserve_value_sequence[:]))
plt.xlabel("number of steps")
plt.ylabel("reserve value")
# plt.title("Shading function obtained by gradient descent \n
# Spline basis, 2 opponents")
plt.savefig('reserve_value.pdf')
plt.show()

In [ ]:
plt.plot(np.asarray(reserve_prices[:]))
plt.xlabel("index")
plt.ylabel("reserve price")
# plt.title("Shading function obtained by gradient descent \n
# Spline basis, 2 opponents")
plt.savefig('reserve_price.pdf')
plt.show()

In [ ]:
plt.plot(np.asarray(revenues[:]))
plt.xlabel("index")
plt.ylabel("revenue player")
plt.title("Evolution of bidder revenue \n Spline basis, " + str(num_oppo) + " opponents")
plt.savefig('revenue_bidder.pdf')
plt.show()

In [ ]:
key_index = np.argmax(np.asarray(revenues[:]))
key_coeffs = all_coeffs[key_index]
test_spline = compute_bidder_shading(these_knots,key_coeffs)

test_x=np.linspace(0.0,1.0,1001)
value_func = np.ones(len(test_x))
for index, my_x in enumerate(test_x):
    value_func[index] = test_spline(my_x)

plt.plot(test_x,value_func)
plt.plot(test_x,test_x,color='r')
plt.xlabel("Value")
plt.ylabel("Bid")
plt.title("Strategy obtained by gradient descent \n Spline basis, " + str(num_oppo) + "opponents")
plt.savefig("Shading_function.pdf")
plt.show()

In [ ]:
test_v_value = compute_v_value_after_shading(these_knots, key_coeffs,max_unif_den, max_unif_cdf)
test_x=np.linspace(0.0,1.0,1001)
value_func = np.ones(len(test_x))
for index, my_x in enumerate(test_x):
    value_func[index] = test_v_value(my_x)

plt.plot(test_x,value_func)
plt.xlabel("value")
plt.ylabel("virtual value function")
plt.axhline(y=0.0, color='r', linestyle='-')
plt.title("Virtual value of Shading function obtained by gradient descent \n Spline basis, " + str(num_oppo) + " opponents")
plt.savefig("virtual_value_spline.pdf")
plt.show()

### Going further down the descent path

In [ ]:
these_knots = np.linspace(.1,.9,10)
key_coeffs = all_coeffs[key_index]
these_coeffs = key_coeffs
# these_coeffs[0]=0
# these_coeffs[1]=0.7
this_x_0 = reserve_value_sequence[key_index] 
num_oppo = 1
num_steps = 500
grad_step = 10**(-2)
#I changed the code slightly there; the reward vector in the lp is now step_size*derivative_direction
start = time.time()
test_2, reserve_value_sequence_2, all_coeffs_2, reserve_prices_2, revenues_2=perform_multiple_gradient_ascent_steps(density_opponents= lambda x: max_unif_den(x,num_oppo), 
                                            cdf_opponents = lambda x: max_unif_cdf(x,num_oppo),
                                                 knots_bidder = these_knots,
                                                 coeffs_bidder_shading = these_coeffs,
                                                 density_bidder = max_unif_den, 
                                                cdf_bidder = max_unif_cdf,
                                                 x_0 = this_x_0, 
                                                assume_v_val_inc=False,
                                        lower_bound=0.0, upper_bound=1.0,
                                        gradient_step_size=grad_step,
                                        abs_max_size_move=1.0,
                                        number_steps=num_steps,
                                        less_than_id_constraint=False,
                                           monotonicity_constraint=False,
                                           brute_force_comp_res_val=True,
                                           test_vals=[0.0,1.0],
                                           my_tol = 10**(-6))
elapsed = time.time() - start
print("Elapsed time", elapsed)

In [ ]:
plt.figure(figsize=(6,6))
#key_index = np.argmax(np.asarray(revenues[:]))
key_coeffs_2 = test_2
test_spline_2 = compute_bidder_shading(these_knots,key_coeffs_2)

test_x=np.linspace(0.0,1.0,1001)
value_func = np.ones(len(test_x))
for index, my_x in enumerate(test_x):
    value_func[index] = test_spline(my_x)

plt.plot(test_x,value_func)
plt.plot(test_x,test_x,color='r')
plt.xlabel("value")
plt.ylabel("shading function")
plt.title("Shading function obtained by gradient descent \n Spline basis, " + str(num_oppo) + "opponents")
#plt.savefig('ShadingFunctionEndGradDescent.png')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
test_v_value_2 = compute_v_value_after_shading(these_knots, key_coeffs_2,max_unif_den, max_unif_cdf)
test_x=np.linspace(0.0,1.0,1001)
value_func_2 = np.ones(len(test_x))
for index, my_x in enumerate(test_x):
    value_func_2[index] = test_v_value_2(my_x)

plt.plot(test_x,value_func_2)
plt.xlabel("value")
plt.ylabel("virtual value function")
plt.axhline(y=0.0, color='r', linestyle='-')
plt.title("Virtual value of Shading function obtained by gradient descent \n Spline basis, " + str(num_oppo) + " opponents")
#plt.savefig('VValueEndGradDescent.png')
plt.show()

In [ ]:
verif_reserve_value_2, revenue_seller_2 = compute_x_0_from_v_value_brute_force(test_v_value_2,
                                         test_spline_2,
                                         cdf_opponents = lambda x: max_unif_cdf(x,num_oppo),
                                         density_bidder = max_unif_den,
                                        test_vals=[0.0,1.0])

In [ ]:
verif_reserve_value_2

In [ ]:
plt.figure(figsize=(6,6))
grid_size = 51
grid = np.linspace(0,1,grid_size)
plt.plot(grid, revenue_seller_2)
plt.xlabel("possible reserve values")
plt.ylabel("revenue seller")
plt.title("Revenue seller for various reserve values")
plt.savefig('RevSellerEndGradDescent.png')
plt.show()